In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages

# =======================================================
# 1. FIXED-LENGTH SPARKLINE FUNCTIONS (using interpolation)
# =======================================================
def fixed_colored_sparkline_html(data, max_points=30):
    """
    Create an HTML sparkline with fixed number of ticks (max_points) using inline CSS colors.
    Data is interpolated to exactly max_points entries.
    """
    data = np.asarray(data)
    if data.size == 0:
        return ""
    if len(data) != max_points:
        x_old = np.linspace(0, 1, num=len(data))
        x_new = np.linspace(0, 1, num=max_points)
        data = np.interp(x_new, x_old, data)
    
    ticks = "▁▂▃▄▅▆▇█"
    mn, mx = data.min(), data.max()
    rng = mx - mn
    if rng == 0:
        chars = [ticks[0]] * max_points
    else:
        chars = [
            ticks[int((d - mn) / (rng + 1e-9) * (len(ticks)-1))]
            for d in data
        ]
    spans = []
    for d, char in zip(data, chars):
        norm = (d - mn) / (rng + 1e-9)
        r = int(255 * (1 - norm))
        g = int(255 * norm)
        b = 0
        color_code = f"\033[38;2;{r};{g};{b}m"
        reset_code = "\033[0m"
        spans.append(f"{color_code}{char}{reset_code}")
    return " ".join(spans)


def fixed_plain_sparkline_csv(data, max_points=30):
    """
    Create a plain Unicode sparkline for CSV output with exactly max_points ticks,
    interpolating the data as necessary.
    """
    data = np.asarray(data)
    if data.size == 0:
        return ""
    if len(data) != max_points:
        x_old = np.linspace(0, 1, num=len(data))
        x_new = np.linspace(0, 1, num=max_points)
        data = np.interp(x_new, x_old, data)
    
    ticks = "▁▂▃▄▅▆▇█"
    mn, mx = data.min(), data.max()
    if mx == mn:
        return " ".join([ticks[0]] * max_points)
    return " ".join(
        ticks[int((d - mn) / (mx - mn + 1e-9) * (len(ticks)-1))]
        for d in data
    )

# =======================================================
# 2. Data Loading & Preparation
# =======================================================
df_wage = pd.read_csv('/Users/nadegelan/Documents/E4S/MA-2/MGT-502/Project ML/Countries_Wages.csv')
df_compare = df_wage.copy()

# Compute log-transformed wages
df_compare["log_estimated_hourly_wage"] = np.log(df_compare["estimated_hourly_wage"])

# =======================================================
# 3. Here Dr.Cashmere Flow computes the OECD Benchmark
# =======================================================
oecd_standard = {
    'Australia', 'Austria', 'Belgium', 'Canada', 'Czechia',
    'Denmark', 'Finland', 'France', 'Germany', 'Greece',
    'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan',
    'Luxembourg', 'Netherlands', 'New Zealand', 'Norway', 'Portugal',
    'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
    'United Kingdom', 'United States'
}
df_oecd_only = df_compare[
    df_compare["country"].isin(oecd_standard) &
    (df_compare["year"] >= 1991) &
    (df_compare["year"] <= 2023)
].copy()

benchmark_by_year = df_oecd_only.groupby("year")["log_estimated_hourly_wage"].mean().reset_index(name="benchmark_log_wage")
overall_benchmark = benchmark_by_year["benchmark_log_wage"].mean()
print("OECD Benchmark by Year (1991-2023):")
print(benchmark_by_year)
print("\nOverall OECD Benchmark (1991-2023):", overall_benchmark)

# =======================================================
# 4. Prepare Data for Convergence Analysis
# =======================================================
df_non_oecd = df_compare[~df_compare["country"].isin(oecd_standard)].copy()
df_non_oecd = pd.merge(df_non_oecd, benchmark_by_year, on="year", how="left")
df_non_oecd["wage_gap"] = df_non_oecd["log_estimated_hourly_wage"] - df_non_oecd["benchmark_log_wage"]

# =======================================================
# 5. Convergence Analysis: If Pipeline Improves, Use It; Else Keep Linear
# =======================================================
extrapolation_year = 2080
convergence_levels = [0.7, 0.8, 0.9]
log_thresholds = {lv: np.log(lv) for lv in convergence_levels}

results_list = []
improvement_list = []

pdf_all = PdfPages(os.path.expanduser("~/Downloads/Convergence_feature_tuning.pdf"))

for country in sorted(df_non_oecd["country"].unique()):
    df_country = df_non_oecd[df_non_oecd["country"] == country].dropna(subset=["wage_gap"]).sort_values("year")
    if len(df_country) < 5:
        continue
    
    # Recenter year for pipeline models
    df_country["year_centered"] = df_country["year"] - df_country["year"].min()
    X = df_country[["year_centered"]].values
    y = df_country["wage_gap"].values
    
    # --------------------------
    # (A) Linear Model (baseline)
    # --------------------------
    a_lin, b_lin = np.polyfit(df_country["year"], y, 1)
    y_linear_pred = a_lin * df_country["year"] + b_lin
    mse_linear = mean_squared_error(y, y_linear_pred)

    # --------------------------
    # (B) Try Pipelines & Compare MSE
    # --------------------------
    pipelines = {
        "Quadratic": Pipeline([
            ("poly", PolynomialFeatures(degree=2, include_bias=False)),
            ("linreg", LinearRegression())
        ]),
        "Cubic": Pipeline([
            ("poly", PolynomialFeatures(degree=3, include_bias=False)),
            ("linreg", LinearRegression())
        ]),
        "SquareRoot": Pipeline([
            ("sqrt", FunctionTransformer(np.sqrt, validate=False)),
            ("linreg", LinearRegression())
        ]),
        # We omit "Linear" pipeline here, as we already have a_lin, b_lin
        # But you could still keep it for clarity.
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline_mse_results = {}
    for name, pipe in pipelines.items():
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        pipeline_mse_results[name] = mean_squared_error(y_test, y_pred)
    # Find pipeline with the smallest MSE
    if len(pipeline_mse_results) > 0:
        best_pipeline_name = min(pipeline_mse_results, key=pipeline_mse_results.get)
        pipeline_best_mse = pipeline_mse_results[best_pipeline_name]
    else:
        best_pipeline_name = None
        pipeline_best_mse = float("inf")
    
    # Compare best pipeline MSE with linear MSE
    if pipeline_best_mse < mse_linear:
        # Pipeline is strictly better
        final_model_name = best_pipeline_name
        final_mse = pipeline_best_mse
        # Fit it on full data
        final_model = pipelines[best_pipeline_name]
        final_model.fit(X, y)
    else:
        # Linear is better (or tie)
        final_model_name = "Linear"
        final_mse = mse_linear
        final_model = None  # We'll just use (a_lin,b_lin) for predictions
    
    improvement_pct = ((mse_linear - final_mse) / mse_linear * 100) if mse_linear != 0 else 0
    
    # --------------------------
    # (C) Compute thresholds for the final model
    # --------------------------
    min_year = df_country["year"].min()
    last_year = df_country["year"].max()
    future_years = np.arange(last_year + 1, extrapolation_year + 1)
    
    def final_model_predict(yrs):
        """Predict wage_gap for an array of raw years using the chosen final model."""
        if final_model_name == "Linear":
            return a_lin * yrs + b_lin
        else:
            # Pipeline needs "year_centered" for input
            centerX = (yrs - min_year).reshape(-1,1)
            return final_model.predict(centerX)
    
    # Build predictions for future
    future_pred = final_model_predict(future_years)
    
    # For threshold crossing, we check the first year from last_year+1 to 2080
    thresholds_final = {}
    for lv, log_thresh in log_thresholds.items():
        # Indices in future_pred where we exceed (>=) the log_thresh
        idx = np.where(future_pred >= log_thresh)[0]
        if len(idx) == 0:
            thresholds_final[lv] = "Not before 2080"
        else:
            # The earliest crossing
            candidate_year = future_years[idx[0]]
            if candidate_year <= last_year:
                thresholds_final[lv] = "Already crossed"
            else:
                thresholds_final[lv] = str(int(candidate_year))
    
    # Replace "Not before 2080" with "X"
    for lv in thresholds_final:
        if thresholds_final[lv] == "Not before 2080":
            thresholds_final[lv] = "X"
    
    # --------------------------
    # (D) Build sparkline, compute R^2 of linear
    # --------------------------
    plain_spark = fixed_plain_sparkline_csv(y, max_points=30)
    colored_spark = fixed_colored_sparkline_html(y, max_points=30)
    r2_val = round(sm.OLS(y, sm.add_constant(df_country["year"])).fit().rsquared, 3)
    
    # --------------------------
    # (E) Build row for summary
    # --------------------------
    row_dict = {
        "country": country,
        "linear_mse": round(mse_linear, 3),
        "linear_slope": round(a_lin, 3),
        "linear_intercept": round(b_lin, 2),  # 2 decimals
        "final_model": final_model_name,
        "final_mse": round(final_mse, 3),
        "r2_linear": r2_val,
        "thresh_70": thresholds_final[0.7],
        "thresh_80": thresholds_final[0.8],
        "thresh_90": thresholds_final[0.9],
        "improvement_pct": round(improvement_pct, 1),
        "sparkline": plain_spark
    }
    results_list.append(row_dict)
    improvement_list.append((country, improvement_pct))
    
    # --------------------------
    # (F) Plotting
    # --------------------------
    # We'll do 2 lines:
    # - Gray line for linear (observed years). Dashed extension for linear future if final != linear
    # - Red line for final model (observed + dashed future)
    
    # Observed time points for plotting
    y_lin_obs = a_lin * df_country["year"] + b_lin
    # Future time points for linear
    y_lin_future = a_lin * future_years + b_lin
    
    # For final model in the observed period:
    def final_pred_observed(yrs):
        if final_model_name == "Linear":
            return y_lin_obs
        else:
            centerX = (yrs - min_year).values.reshape(-1, 1)
            return final_model.predict(centerX)
    
    y_final_obs = final_pred_observed(df_country["year"])
    # Future final
    y_final_future = final_model_predict(future_years)
    
    # Begin plotting
    plt.figure(figsize=(7, 4))
    # Observed data
    plt.scatter(df_country["year"], y, label="Observed Gap", facecolors='none', edgecolors='gray')
    
    # Always show linear in gray for the observed period
    if final_model_name != "Linear":
        plt.plot(df_country["year"], y_lin_obs, color="gray", label=f"Linear (MSE={mse_linear:.3f})")
        plt.plot(future_years, y_lin_future, color="gray", linestyle="--", alpha=0.3, label="Linear Extra.")
    
    # Plot final model in red
    plt.plot(df_country["year"], y_final_obs, color="red", label=f"{final_model_name} (MSE={final_mse:.3f})")
    # Add a dashed red line for future
    plt.plot(future_years, y_final_future, color="red", linestyle="--", alpha=0.7, label=f"{final_model_name} Extra.")
    
    # Horizontal reference line (0)
    plt.axhline(0, color="black", linestyle="--", linewidth=1)
    
    # Optionally, we can also plot a threshold line, e.g. 80%
    plt.axhline(log_thresholds[0.8], color="blue", linestyle=":", label="80% threshold")
    
    plt.title(country)
    plt.xlabel("Year")
    plt.ylabel("Wage Gap (log scale)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    pdf_all.savefig()
    plt.close()
    
    # Console summary
    print(f"[{country}] LinMSE={mse_linear:.3f}; Best={final_model_name} (MSE={final_mse:.3f}); "
          f"Improv={improvement_pct:.1f}% | Spark: {colored_spark}")

pdf_all.close()

# =======================================================
# 6. Compile and Save Summary Table
# =======================================================
df_summary = pd.DataFrame(results_list)

# Helper to parse final model threshold_80 for sorting
def parse_threshold(val):
    if val == "Already crossed":
        return -9999
    elif val == "X":
        return 9999
    else:
        try:
            return int(val)
        except:
            return 9999

df_summary["sort_key"] = df_summary["thresh_80"].apply(parse_threshold)
df_summary.sort_values(by="sort_key", inplace=True)
df_summary.drop(columns=["sort_key"], inplace=True)

summary_csv = os.path.expanduser("~/Downloads/Convergence_Extrapolation_Result.csv")
df_summary.to_csv(summary_csv, index=False)

print("\nConvergence Summary for Non-OECD Countries (Sorted by Final Model's 80% Threshold):")
cols_to_show = [
    "country", "linear_mse", "linear_slope", "linear_intercept", 
    "final_model", "final_mse", "thresh_80", 
    "r2_linear", "improvement_pct", "sparkline"
]
print(df_summary[cols_to_show])
print(f"Convergence summary saved to {summary_csv}")

# =======================================================
# 7. Produce Additional Plots for Top 10 Countries by Improvement
# =======================================================
improvement_sorted = sorted(improvement_list, key=lambda x: x[1], reverse=True)
top10 = [c for c, _ in improvement_sorted[:10]]

pdf_top10 = PdfPages(os.path.expanduser("~/Downloads/Top_10_improvement_plots.pdf"))
for country in top10:
    df_country = df_non_oecd[df_non_oecd["country"] == country].dropna(subset=["wage_gap"]).sort_values("year")
    if len(df_country) < 5:
        continue
    
    # Same approach: first get linear stats
    a_lin, b_lin = np.polyfit(df_country["year"], df_country["wage_gap"], 1)
    mse_linear = mean_squared_error(df_country["wage_gap"], (a_lin * df_country["year"] + b_lin))
    
    # Then see if a pipeline does better
    X = (df_country["year"] - df_country["year"].min()).values.reshape(-1,1)
    y = df_country["wage_gap"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    pipelines = {
        "Quadratic": Pipeline([
            ("poly", PolynomialFeatures(degree=2, include_bias=False)),
            ("linreg", LinearRegression())
        ]),
        "Cubic": Pipeline([
            ("poly", PolynomialFeatures(degree=3, include_bias=False)),
            ("linreg", LinearRegression())
        ]),
        "SquareRoot": Pipeline([
            ("sqrt", FunctionTransformer(np.sqrt, validate=False)),
            ("linreg", LinearRegression())
        ])
    }
    pipeline_mse_results = {}
    for name, pipe in pipelines.items():
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        pipeline_mse_results[name] = mean_squared_error(y_test, y_pred)
    
    if pipeline_mse_results:
        best_pipeline = min(pipeline_mse_results, key=pipeline_mse_results.get)
        best_pipeline_mse = pipeline_mse_results[best_pipeline]
    else:
        best_pipeline = None
        best_pipeline_mse = float("inf")
    
    if best_pipeline_mse < mse_linear:
        final_model_name = best_pipeline
        final_mse = best_pipeline_mse
        final_model = pipelines[best_pipeline]
        final_model.fit(X, y)
    else:
        final_model_name = "Linear"
        final_mse = mse_linear
        final_model = None  # We'll just use a_lin,b_lin
    
    # Plot
    min_yr = df_country["year"].min()
    last_yr = df_country["year"].max()
    future_yrs = np.arange(last_yr+1, extrapolation_year+1)
    
    def predict_final(yrs_array):
        if final_model_name == "Linear":
            return a_lin * yrs_array + b_lin
        else:
            cX = (yrs_array - min_yr).reshape(-1,1)
            return final_model.predict(cX)
    
    y_obs_lin = a_lin * df_country["year"] + b_lin
    y_future_lin = a_lin * future_yrs + b_lin
    
    y_obs_final = predict_final(df_country["year"].values)
    y_future_final = predict_final(future_yrs)
    
    plt.figure(figsize=(7, 4))
    # Observed points
    plt.scatter(df_country["year"], y, label="Observed", facecolors='none', edgecolors='gray')
    
    # Show linear in gray only if final_model != "Linear"
    if final_model_name != "Linear":
        plt.plot(df_country["year"], y_obs_lin, color="gray", label=f"Linear(MSE={mse_linear:.3f})")
        plt.plot(future_yrs, y_future_lin, color="gray", linestyle="--", alpha=0.3, label="Lin Extra.")
    
    # Show final in red
    plt.plot(df_country["year"], y_obs_final, color="red", label=f"{final_model_name}(MSE={final_mse:.3f})")
    plt.plot(future_yrs, y_future_final, color="red", linestyle="--", alpha=0.7, label=f"{final_model_name} Extra.")
    
    plt.axhline(0, color="black", linestyle="--", linewidth=1)
    plt.title(country)
    plt.xlabel("Year")
    plt.ylabel("Wage Gap (log scale)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    pdf_top10.savefig()
    plt.close()

pdf_top10.close()
print("Top 10 improvement plots saved to '~/Downloads/Top_10_improvement_plots.pdf'")


OECD Benchmark by Year (1991-2023):
    year  benchmark_log_wage
0   1991            3.126180
1   1992            3.138700
2   1993            3.155430
3   1994            3.175427
4   1995            3.188173
5   1996            3.210003
6   1997            3.232294
7   1998            3.252553
8   1999            3.276625
9   2000            3.306530
10  2001            3.330462
11  2002            3.349354
12  2003            3.371119
13  2004            3.390709
14  2005            3.414933
15  2006            3.434680
16  2007            3.455064
17  2008            3.462639
18  2009            3.471444
19  2010            3.482578
20  2011            3.493145
21  2012            3.501092
22  2013            3.505581
23  2014            3.508230
24  2015            3.510995
25  2016            3.522465
26  2017            3.539094
27  2018            3.555464
28  2019            3.568866
29  2020            3.705075
30  2021            3.729446
31  2022            3.720074
32  202

In [14]:
df_wage['iso_code'].unique()

array(['AGO', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN',
       'BFA', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BOL', 'BRA', 'BRB',
       'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CMR', 'COL', 'CPV',
       'CRI', 'CYP', 'CZE', 'DEU', 'DNK', 'DOM', 'ECU', 'EGY', 'ESP',
       'EST', 'FIN', 'FJI', 'FRA', 'GAB', 'GBR', 'GEO', 'GIN', 'GRC',
       'GTM', 'HKG', 'HND', 'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'IRN',
       'IRQ', 'ISL', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ',
       'KOR', 'KWT', 'LAO', 'LBN', 'LKA', 'LSO', 'LUX', 'LVA', 'MAC',
       'MAR', 'MDA', 'MEX', 'MKD', 'MLT', 'MNG', 'MOZ', 'MRT', 'MUS',
       'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NZL', 'OMN',
       'PER', 'PHL', 'POL', 'PRT', 'PRY', 'QAT', 'ROU', 'RUS', 'RWA',
       'SAU', 'SDN', 'SEN', 'SGP', 'SLE', 'SRB', 'SUR', 'SVK', 'SVN',
       'SWE', 'SWZ', 'TCD', 'TGO', 'THA', 'TJK', 'TTO', 'TUN', 'TZA',
       'UKR', 'URY', 'USA', 'UZB', 'ZAF', 'ZMB', 'ZWE'], dtype=object)

# Data Collection

In [29]:

link='https://raw.githubusercontent.com/baertsch/MGT-502-ML-Project/e8c3d4028ed71abb4d4cb9d3d5273dc0b2bfad1c/'

## Trade Openness Index

In [ ]:
import pandas as pd
import requests

# Fetch the data.
# Trade Openness Index (TOI)
df_TOI = pd.read_csv("https://ourworldindata.org/grapher/trade-as-share-of-gdp.csv?v=1&csvType=full&useColumnShortNames=false", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata_TOI = requests.get("https://ourworldindata.org/grapher/trade-as-share-of-gdp.metadata.json?v=1&csvType=full&useColumnShortNames=false").json()

## Educational Attainment

In [30]:
df_edu = pd.read_csv(
    link+'educational_attainment.csv',
    skiprows=4  # Skip the first 4 rows of metadata
)

In [32]:
df_edu.isna().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              266
                 ... 
2021              147
2022              145
2023              213
2024              266
Unnamed: 69       266
Length: 70, dtype: int64